# Assignment 5 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [1]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, make_scorer
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [2]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [3]:
# word net installation:

# unmark if you want to use and need to install
# !pip install wn
# !python -m wn download omw-he:1.4

In [ ]:
# word net import:

# unmark if you want to use:
import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [5]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
# !pip install hebrew_tokenizer

In [6]:
# Hebrew tokenizer import:

# unmark if you want to use:
import hebrew_tokenizer as ht

/Users/yairnagar/Downloads/assignment5-text_analysis


### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [9]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [10]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

### Your implementation:
Write your code solution in the following code-cells

In [11]:
# YOUR CODE HERE


In [12]:
def clean_all_text(text):
    text = re.sub(r'[,.;:?!/"()\-a-zA-Z]', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    return text

In [13]:
df_train['story'] = df_train['story'].apply(clean_all_text)
X = df_train['story']
y = df_train['gender']

In [14]:
vectorizer = CountVectorizer(ngram_range=(1, 3), min_df=3, max_features=11560)
df_train_vec = vectorizer.fit_transform(X)

models = [
    {
        'name': 'MultinomialNB',
        'model': MultinomialNB(),
        'params': {'alpha': [0.1, 0.5, 1.0, 1.5, 2.0], 'fit_prior': [True, False]}
    },
    {
        'name': 'Decision Tree',
        'model': DecisionTreeClassifier(),
        'params': {'max_depth': [None, 5, 10, 20, 50, 100]}
    },
    {
        'name': 'K-Nearest Neighbors',
        'model': KNeighborsClassifier(),
        'params': {'n_neighbors': [3, 5, 7, 9, 11, 13], 'weights': ['uniform', 'distance']}
    },
    {
        'name': 'LinearSVC',
        'model': LinearSVC(),
        'params': {'C': [0.01, 0.1, 1.0, 10.0, 100.0], 'penalty': ['l1', 'l2'], 'dual': [True, False]}
    },
    {
        'name': 'Perceptron',
        'model': Perceptron(),
        'params': {'alpha': [0.0001, 0.001, 0.01, 0.1, 1.0], 'penalty': ['l1', 'l2'], 'fit_intercept': [True, False]}
    }
]

f1_scorer = make_scorer(f1_score, average='macro', greater_is_better=True)


In [15]:
for model_info in models:
    model = model_info['model']
    name = model_info['name']
    params = model_info['params']
    
    grid_search = GridSearchCV(model, params, scoring=f1_scorer, cv=5)
    grid_search.fit(df_train_vec, y)
    
    best_model = grid_search.best_estimator_
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_

    print(name)
    print("Best Parameters:", best_params)
    print("Best F1 Score:", best_score)
    
    if best_score >= 0.70:
        print(name, "is a good model for predicting gender.")

GridSearchCV(cv=5, estimator=MultinomialNB(),
             param_grid={'alpha': [0.1, 0.5, 1.0, 1.5, 2.0],
                         'fit_prior': [True, False]},
             scoring=make_scorer(f1_score, average=macro))

MultinomialNB
Best Parameters: {'alpha': 1.0, 'fit_prior': False}
Best F1 Score: 0.7210094408050581
MultinomialNB is a good model for predicting gender.


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [None, 5, 10, 20, 50, 100]},
             scoring=make_scorer(f1_score, average=macro))

Decision Tree
Best Parameters: {'max_depth': 20}
Best F1 Score: 0.611608520010724


GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [3, 5, 7, 9, 11, 13],
                         'weights': ['uniform', 'distance']},
             scoring=make_scorer(f1_score, average=macro))

K-Nearest Neighbors
Best Parameters: {'n_neighbors': 3, 'weights': 'distance'}
Best F1 Score: 0.5431281898340441


GridSearchCV(cv=5, estimator=LinearSVC(),
             param_grid={'C': [0.01, 0.1, 1.0, 10.0, 100.0],
                         'dual': [True, False], 'penalty': ['l1', 'l2']},
             scoring=make_scorer(f1_score, average=macro))

LinearSVC
Best Parameters: {'C': 0.1, 'dual': True, 'penalty': 'l2'}
Best F1 Score: 0.7031550120142678
LinearSVC is a good model for predicting gender.


GridSearchCV(cv=5, estimator=Perceptron(),
             param_grid={'alpha': [0.0001, 0.001, 0.01, 0.1, 1.0],
                         'fit_intercept': [True, False],
                         'penalty': ['l1', 'l2']},
             scoring=make_scorer(f1_score, average=macro))

Perceptron
Best Parameters: {'alpha': 0.0001, 'fit_intercept': False, 'penalty': 'l1'}
Best F1 Score: 0.6676659892024306


In [16]:
best_model.fit(df_train_vec, y)

X_new_vect = vectorizer.transform(df_test['story'])

Perceptron(fit_intercept=False, penalty='l1')

In [17]:
y_new_pred = best_model.predict(X_new_vect)

df_predicted = pd.DataFrame({'test_example_id': df_test['test_example_id'], 'predicted_category': y_new_pred})
df_predicted

,test_example_id,predicted_category
0,0,m
1,1,m
2,2,m
3,3,m
4,4,f
...,...,...
318,318,m
319,319,m
320,320,m
321,321,m


### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [18]:
df_predicted.to_csv('classification_results.csv',index=False)